RAG - Loader(UpstageDocumentParseLoader)

In [ ]:
# 로더 실행
from dotenv import load_dotenv
from langchain_upstage import UpstageDocumentParseLoader

file_path = "./test_modified.pdf"

loader = UpstageDocumentParseLoader(
    file_path,
    split="page",  # 페이지별로 분할
    output_format="markdown",  # 텍스트 형태로 출력
    ocr="auto",  # 자동 OCR 사용
    coordinates=True,  # 좌표 정보 포함
)


docs = loader.load()

In [ ]:
# print(docs[21].page_content)
print(docs[0])

Rag - Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#  문서 자르기 (AI가 읽기 쉬운 크기로 쪼개기)
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
docs_splitter = splitter.split_documents(docs) 

Rag - Embedding

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True},
)

Rag - Vector Store

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

# 올바른 변수명 사용
vectorstore = FAISS.from_documents(
    documents=docs_splitter,  # 기존 변수명 그대로 사용
    embedding=hf_embeddings,  # 임베딩 모델
)

Rag - retriever

In [25]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 유사도 검색
    search_kwargs={"k": 5},  # 상위 5개 결과 반환
)

Rag - Formatting / Prompt

In [26]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.chat_history import InMemoryChatMessageHistory

In [ ]:
# LLM 설정 (스트리밍 활성화)
llm = ChatAnthropic(model="claude-3-5-haiku-20241022", temperature=0, streaming=True)

# 대화 기록 저장소
chat_history = InMemoryChatMessageHistory()


# 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# RAG + 멀티턴 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """너는 친절한 한국어 AI 비서야. 
        제공된 문서 내용(context)과 이전 대화 내용을 참고해서 질문에 답해.
        반드시 한국어로만 대답하고, 문서에 없는 내용은 일반 지식으로 보완해서 답변해.
        
        참고 문서:
        {context}
        """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

# RAG 체인 구성 (멀티턴 지원)
rag_multiturn_chain = (
    {"context": retriever | format_docs, "input": lambda x: x["input"]}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
def limit_chat_history(history_obj, max_turns=3):
    """최근 'max_turns'번의 사용자-AI 대화만 남기기"""
    max_messages = max_turns * 2
    if len(history_obj.messages) > max_messages:
        deleted_count = len(history_obj.messages) - max_messages  
        for _ in range(deleted_count):
            history_obj.messages.pop(0)


def stream_rag_multiturn_response(user_input):
    """RAG + 멀티턴 대화를 스트리밍으로 처리"""

    # 사용자 입력 저장
    chat_history.add_user_message(user_input)

    # 히스토리 제한 적용
    limit_chat_history(chat_history)

    print(f"🙋 사용자: {user_input}")
    print("🤖 AI: ", end="", flush=True)

    response = ""

    try:
        response = ""
        # # 프롬프트에 히스토리 직접 바인딩
        prompt_with_history = prompt.partial(chat_history=chat_history.messages[:-1])

        # 체인 재구성
        chain_with_history = (
            {"context": retriever | format_docs, "input": lambda x: x}
            | prompt_with_history
            | llm
            | StrOutputParser()
        )

        # 스트리밍으로 응답 받기
        for chunk in chain_with_history.stream(user_input):
            print(chunk, end="", flush=True)
            response += chunk

        print()  # 줄바꿈

        # AI 응답 저장
        chat_history.add_ai_message(response)

        return response

    except Exception as e:
        error_msg = f"❌ 오류 발생: {str(e)}"
        print(error_msg)
        return error_msg

Rag - Streaming

In [ ]:
# 대화형 인터페이스
def interactive_chat():
    """대화형 채팅 인터페이스"""
    print("=" * 60)
    print("🤖 RAG 챗봇 시작!")
    print("💡 '종료' 입력 시 대화를 끝냅니다.")
    print("=" * 60)


    try:
        while True:
            try:
                user_input = input("\n🙋 사용자: ").strip()

                if user_input.lower() == "종료":
                    print("👋 채팅을 종료합니다!")
                    break

                
                # stream_rag_multiturn_response(user_input)

                # 스트리밍 응답 처리
                stream_rag_multiturn_response(user_input)

            except KeyboardInterrupt:
                print("\n채팅을 종료합니다!")
                break

    finally:
        
        chat_history.clear()

Rag - QnA

In [33]:
if __name__ == "__main__":
    interactive_chat()  # 대화형 모드 실행

🤖 RAG 챗봇 시작!
💡 '종료' 입력 시 대화를 끝냅니다.
🙋 사용자: 안녕
🤖 AI: 안녕하세요! 아주대학교에 대해 궁금한 점이 있으시면 언제든 물어보세요. 제가 최선을 다해 답변해드리겠습니다.
👋 채팅을 종료합니다!
